Credits:


*   Poopmaster for something
*   Chat gpt for yes
*   [Pyannote](https://github.com/pyannote/pyannote-audio)


In [ ]:
#@title Download pyannote and connect to drive
!pip install pyannote-audio pyannote-core pydub
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#@title Diarization
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.audio import Pipeline
from pydub import AudioSegment
import os
import torch

HUGGINGFACE_ACCESS_TOKEN_GOES_HERE = "" #@param {type:"string"}
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HUGGINGFACE_ACCESS_TOKEN_GOES_HERE
)

# Check if GPU is available
if torch.cuda.is_available():
    # Send pipeline to GPU
    pipeline.to(torch.device("cuda"))
    print("Pipeline sent to GPU")


# Function to perform speaker diarization and export speaker audio segments
def export_speaker_audio(input_audio_path, output_dir):
    # Load pretrained speaker diarization pipeline
    diarization_pipeline = SpeakerDiarization.from_pretrained("pyannote/speaker-diarization-3.1")

    # Apply diarization to the audio file
    diarization_result = diarization_pipeline({'uri': 'filename', 'audio': input_audio_path})

    # Create output directory to store individual speaker audio files
    os.makedirs(output_dir, exist_ok=True)

    for i, (turn, _, speaker) in enumerate(diarization_result.itertracks(yield_label=True)):
        speaker_audio = AudioSegment.from_wav(input_audio_path)[int(turn.start * 1000):int(turn.end * 1000)]
        output_path = os.path.join(output_dir, f'speaker_{speaker}_{i+1}.wav')
        speaker_audio.export(output_path, format="wav")
        print(f"Exported speaker {speaker} audio to: {output_path}")

# Input audio file path
input_audio_path = '/content/drive/MyDrive/test.wav' #@param {type:"string"}

# Output directory for individual speaker audio files
output_directory = '/content/speaker_audio'

# Perform speaker diarization and export speaker audio segments
export_speaker_audio(input_audio_path, output_directory)



In [ ]:
#@title Export to drive
import os
import shutil
from google.colab import drive

# Define the path to your audio files
audio_files_path = '/content/speaker_audio'

# Create a function to extract the speaker number from the filename
def extract_speaker_number(filename):
    return int(filename.split('_')[2])

# Create a function to move the files to the corresponding folders
def move_files_to_folders():
    for filename in os.listdir(audio_files_path):
        if filename.endswith('.wav'):
            speaker_number = extract_speaker_number(filename)
            folder_path = os.path.join(audio_files_path, f'speaker_{speaker_number}')

            # Create the folder if it doesn't exist
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # Move the file to the corresponding folder
            shutil.move(os.path.join(audio_files_path, filename), os.path.join(folder_path, filename))
            print(f'Moved {filename} to {folder_path}')

# Call the function to move files to folders
move_files_to_folders()

# Source directory
src_directory = "/content/speaker_audio"

# Destination directory in Google Drive
dst_directory = "/content/drive/My Drive/Audio_Diarization"

# Create the destination directory if it doesn't exist
shutil.copytree(src_directory, dst_directory)
